In [1]:
# Load model directly
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import torch 
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny").to(device)


In [2]:
from PIL import Image
image = Image.open("../examp.png")

text_queries = ["bench"]
inputs = processor(text=text_queries, images=image, return_tensors="pt").to(device)

In [15]:
gt_1 = {"class_labels" : torch.LongTensor(size=(900,)),
        "boxes" : torch.FloatTensor(size=(900,4))}

gt_1["class_labels"][0]

tensor(94711973766344)

In [3]:


outputs = model(**inputs,return_dict=True)
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_grounded_object_detection(outputs, inputs.input_ids, box_threshold = 0.4, text_threshold = 0.3, target_sizes=target_sizes)[0]

#loss = F.nll_loss(output, target.unsqueeze(0))



In [4]:
outputs.logits.size()

torch.Size([1, 900, 256])

In [5]:
from PIL import ImageDraw
draw = ImageDraw.Draw(image)

scores = results["scores"]
labels = results["labels"]
boxes = results["boxes"]

for box, score, label in zip(boxes, scores, labels):
    xmin, ymin, xmax, ymax = box
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{text_queries[0]}: {round(score.item(),2)}", fill="white")

In [6]:
results

{'scores': tensor([0.4015], device='cuda:0', grad_fn=<IndexBackward0>),
 'labels': ['bench'],
 'boxes': tensor([[ 65.3624,  98.6677, 511.7585, 341.5509]], device='cuda:0',
        grad_fn=<IndexBackward0>)}